# Convert RBP gene name into Emsembl and entrez gene id and save to file

- to generate RBP gene/protein id mapper for later things

In [1]:
import pandas as pd
eclip_bam = pd.read_csv('ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifesthg38.txt', sep = '\t', header= 0)
rbp = eclip_bam['RBP'].unique()

In [2]:
import mygene
mg = mygene.MyGeneInfo()
rbp_id = mg.querymany(rbp, scopes = 'symbol', species = 9606, fields = 'all', as_dataframe =True)

querying 1-150...done.
Finished.


/home/hsher/miniconda3/lib/python3.7/site-packages/biothings_client/base.py:143: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize(obj)


2 input query terms found no hit:
	['TROVE2', 'AARS']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


# cleaning ensembl protein and transcript ID

In [3]:
def total_ensp(rbp):
    ''' merge two columns with ensembl protein id'''
    all_ensp = set()
    if type(rbp_id.loc[rbp, 'ensembl.protein']) != float:
        all_ensp.update(rbp_id.loc[rbp, 'ensembl.protein'])
    
    if type(rbp_id.loc[rbp, 'ensembl.translation']) != float:
        all_ensp.update(set([i['protein'] for i in rbp_id.loc[rbp, 'ensembl.translation']]))
    if type(rbp_id.loc[rbp, 'ensembl']) != float:
        for i in rbp_id.loc[rbp, 'ensembl']:
            all_ensp.update(set(i['protein']))
    return(all_ensp)

In [4]:
indices = []
for i in rbp_id.index:
    indices.append(total_ensp(i))
rbp_id['esnp'] = indices

In [5]:
def total_enst(rbp):
    all_ensp = set()
    if type(rbp_id.loc[rbp, 'ensembl.transcript']) != float:
        all_ensp.update(rbp_id.loc[rbp, 'ensembl.transcript'])
    
    
    if type(rbp_id.loc[rbp, 'ensembl']) != float:
        for i in rbp_id.loc[rbp, 'ensembl']:
            all_ensp.update(set(i['transcript']))
    return(all_ensp)

In [6]:
indices = []
for i in rbp_id.index:
    indices.append(total_enst(i))
rbp_id['esnt'] = indices

# Extract GO annotation

In [7]:
rbp_id[['go.BP.id', 'go.CC.id', 'go.MF.id']].count()

go.BP.id    5
go.CC.id    2
go.MF.id    4
dtype: int64

In [8]:
rbp_id[['go.BP.term', 'go.CC.term', 'go.MF.term']].count()

go.BP.term    5
go.CC.term    2
go.MF.term    4
dtype: int64

In [9]:
rbp_id['go.BP.term'].value_counts()

rRNA processing                               2
RNA processing                                1
regulation of transcription, DNA-templated    1
RNA metabolic process                         1
Name: go.BP.term, dtype: int64

In [18]:
# strongly suspect it is not complete
gaf = pd.read_csv('goa_human.gaf', sep = '\t', comment = '!', header = None)
gaf.loc[gaf[2].isin(rbp_id.index)]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
7563,UniProtKB,O00425,IGF2BP3,NaN,GO:0003723,PMID:22658674,HDA,NaN,F,Insulin-like growth factor 2 mRNA-binding prot...,IGF2BP3|IMP3|KOC1|VICKZ3,protein,taxon:9606,20140203,UniProt,NaN,NaN
7564,UniProtKB,O00425,IGF2BP3,NaN,GO:0003723,PMID:22681889,HDA,NaN,F,Insulin-like growth factor 2 mRNA-binding prot...,IGF2BP3|IMP3|KOC1|VICKZ3,protein,taxon:9606,20140203,UniProt,NaN,NaN
7565,UniProtKB,O00425,IGF2BP3,NaN,GO:0003730,PMID:20080952,IDA,NaN,F,Insulin-like growth factor 2 mRNA-binding prot...,IGF2BP3|IMP3|KOC1|VICKZ3,protein,taxon:9606,20100401,UniProt,NaN,NaN
7566,UniProtKB,O00425,IGF2BP3,NaN,GO:0005515,PMID:17289661,IPI,UniProtKB:Q9NZI8,F,Insulin-like growth factor 2 mRNA-binding prot...,IGF2BP3|IMP3|KOC1|VICKZ3,protein,taxon:9606,20091002,UniProt,NaN,NaN
7567,UniProtKB,O00425,IGF2BP3,NaN,GO:0005634,GO_REF:0000044,IEA,UniProtKB-SubCell:SL-0191,C,Insulin-like growth factor 2 mRNA-binding prot...,IGF2BP3|IMP3|KOC1|VICKZ3,protein,taxon:9606,20200222,UniProt,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498566,UniProtKB,P26196,DDX6,NaN,GO:0000932,PMID:21873635,IBA,FB:FBgn0004419|MGI:MGI:104976|PANTHER:PTN00277...,C,Probable ATP-dependent RNA helicase DDX6,DDX6|HLR2|RCK,protein,taxon:9606,20190430,GO_Central,NaN,NaN
498818,UniProtKB,P26368,U2AF2,NaN,GO:0089701,PMID:21873635,IBA,FB:FBgn0005411|PANTHER:PTN000566979|PomBase:SP...,C,Splicing factor U2AF 65 kDa subunit,U2AF2|U2AF65,protein,taxon:9606,20170228,GO_Central,NaN,NaN
498931,UniProtKB,P51114,FXR1,NaN,GO:0005737,PMID:21873635,IBA,FB:FBgn0028734|MGI:MGI:104860|MGI:MGI:95564|PA...,C,Fragile X mental retardation syndrome-related ...,FXR1,protein,taxon:9606,20170228,GO_Central,NaN,NaN
499159,UniProtKB,O43172,PRPF4,NaN,GO:0000398,PMID:21873635,IBA,PANTHER:PTN000457233|SGD:S000006382|UniProtKB:...,P,U4/U6 small nuclear ribonucleoprotein Prp4,PRPF4|PRP4,protein,taxon:9606,20170228,GO_Central,NaN,NaN


In [36]:
goterms = pd.read_csv('rbp_go.txt', sep = '\t').loc[:6317, ['GO term', 'Go term description']].drop_duplicates()
goterms = goterms.loc[goterms['GO term'].isin(gaf[4])]

In [40]:
goterms.set_index('GO term')['Go term description']

GO term
GO:2000766    negative regulation of cytoplasmic translation
GO:0006397                                   mRNA Processing
GO:0005515                                   protein binding
GO:0071456                      cellular response to hypoxia
GO:0000166                                nucleotide binding
                                   ...                      
GO:0048644                        muscle organ morphogenesis
GO:0031581                            hemidesmosome assembly
GO:0048666                                neuron development
GO:0007638                           mechanosensory behavior
GO:0030036                   actin cytoskeleton organization
Name: Go term description, Length: 955, dtype: object

In [41]:
gaf[4].map(goterms.set_index('GO term')['Go term description'].to_dict())

0                                  DNA binding
1                                      cytosol
2                                          NaN
3                            metal ion binding
4                                          NaN
                          ...                 
499481                   synaptic transmission
499482                                     NaN
499483                                 cytosol
499484                                     NaN
499485    DNA-directed RNA polymerase activity
Name: 4, Length: 499486, dtype: object

In [47]:
# https://github.com/tanghaibao/goatools/blob/master/notebooks/cell_cycle.ipynb
from goatools.base import download_ncbi_associations
gene2go = download_ncbi_associations()
from goatools.associations import read_ncbi_gene2go

go2geneids_human = read_ncbi_gene2go("gene2go", taxids=[9606], namespace='BP', go2geneids=True)
print("{N} GO terms associated with human NCBI Entrez GeneIDs".format(N=len(go2geneids_human)))

from goatools.go_search import GoSearch

srchhelp = GoSearch("go-basic.obo", go2items=go2geneids_human)
import re

# Compile search pattern for 'cell cycle'
cell_cycle_all = re.compile(r'RNA', flags=re.IGNORECASE)
cell_cycle_not = re.compile(r'cell cycle.independent', flags=re.IGNORECASE)

  EXISTS: gene2go
DEPRECATED read_ncbi_gene2go: USE Gene2GoReader FROM goatools.anno.genetogo_reader
DEPRECATED read_ncbi_gene2go CALLED FROM: <ipython-input-47-d85e54d15dcc> BY <module>
HMS:0:00:05.465039 336,356 annotations, 20,586 genes, 18,410 GOs, 1 taxids READ: gene2go 
12385 IDs in loaded association branch, BP
12385 GO terms associated with human NCBI Entrez GeneIDs
go-basic.obo: fmt(1.2) rel(2020-05-02) 47,240 GO Terms; optional_attrs(comment def relationship synonym xref)


In [48]:
fout_allgos = "cell_cycle_gos_human.log" 
with open(fout_allgos, "w") as log:
    # Search for 'cell cycle' in GO terms
    gos_cc_all = srchhelp.get_matching_gos(cell_cycle_all, prt=log)
    # Find any GOs matching 'cell cycle-independent' (e.g., "lysosome")
    gos_no_cc = srchhelp.get_matching_gos(cell_cycle_not, gos=gos_cc_all, prt=log)
    # Remove GO terms that are not "cell cycle" GOs
    gos = gos_cc_all.difference(gos_no_cc)
    # Add children GOs of cell cycle GOs
    gos_all = srchhelp.add_children_gos(gos)
    # Get Entrez GeneIDs for cell cycle GOs
    geneids = srchhelp.get_items(gos_all)
print("{N} human NCBI Entrez GeneIDs related to 'RNA' found.".format(N=len(geneids)))

14045 human NCBI Entrez GeneIDs related to 'RNA' found.


In [63]:
from goatools.test_data.genes_NCBI_9606_ProteinCoding import GENEID2NT
symbols = []

for geneid in geneids: # geneids associated with cell-cycle
    nt = GENEID2NT.get(geneid, None)
    if nt is not None:
        print("{Symbol:<10} {desc}".format(
                Symbol = nt.Symbol, 
                desc = nt.description))
        symbols.append(nt.Symbol)

A1BG       alpha-1-B glycoprotein
A2M        alpha-2-macroglobulin
SERPINA3   serpin peptidase inhibitor, clade A (alpha-1 antiproteinase, antitrypsin), member 3
AANAT      aralkylamine N-acetyltransferase
AARS       alanyl-tRNA synthetase
ABAT       4-aminobutyrate aminotransferase
ABCA1      ATP binding cassette subfamily A member 1
ABCA2      ATP binding cassette subfamily A member 2
SDE2       SDE2 telomere maintenance homolog (S. pombe)
ABCB7      ATP binding cassette subfamily B member 7
ABCF1      ATP binding cassette subfamily F member 1
ABCA4      ATP binding cassette subfamily A member 4
ABL1       ABL proto-oncogene 1, non-receptor tyrosine kinase
AOC1       amine oxidase, copper containing 1
ABL2       ABL proto-oncogene 2, non-receptor tyrosine kinase
ABO        ABO blood group (transferase A, alpha 1-3-N-acetylgalactosaminyltransferase; transferase B, alpha 1-3-galactosyltransferase)
ABR        active BCR-related
ACAA1      acetyl-CoA acyltransferase 1
ACACA      acetyl-C

CFI        complement factor I
IFI16      interferon, gamma-inducible protein 16
IFI27      interferon, alpha-inducible protein 27
IFI35      interferon induced protein 35
SP110      SP110 nuclear body protein
IFIT2      interferon induced protein with tetratricopeptide repeats 2
IFIT1      interferon induced protein with tetratricopeptide repeats 1
IFIT3      interferon induced protein with tetratricopeptide repeats 3
UBLCP1     ubiquitin like domain containing CTD phosphatase 1
IFNA1      interferon, alpha 1
IFNA2      interferon, alpha 2
IFNA4      interferon, alpha 4
IFNA5      interferon, alpha 5
IFNA6      interferon, alpha 6
IFNA7      interferon, alpha 7
IFNA8      interferon, alpha 8
IFNA10     interferon, alpha 10
IFNA13     interferon, alpha 13
IFNA14     interferon, alpha 14
IFNA16     interferon, alpha 16
IFNA17     interferon, alpha 17
IFNA21     interferon, alpha 21
IFNAR1     interferon alpha and beta receptor subunit 1
IFNAR2     interferon alpha and beta receptor subu

ARID3C     AT-rich interaction domain 3C
RPP25L     ribonuclease P/MRP 25kDa subunit-like
ZNF91      zinc finger protein 91
ZNF90      zinc finger protein 90
ZNF99      zinc finger protein 99
SHOC2      SHOC2 leucine-rich repeat scaffold protein
ZNF222     zinc finger protein 222
ADAM12     ADAM metallopeptidase domain 12
ZNF124     zinc finger protein 124
MKRN3      makorin ring finger protein 3
ZNF131     zinc finger protein 131
ZNF132     zinc finger protein 132
ZNF133     zinc finger protein 133
ZNF134     zinc finger protein 134
ZNF135     zinc finger protein 135
ZNF136     zinc finger protein 136
ZNF138     zinc finger protein 138
ZNF140     zinc finger protein 140
CHCHD10    coiled-coil-helix-coiled-coil-helix domain containing 10
ZNF142     zinc finger protein 142
ZNF143     zinc finger protein 143
PCGF2      polycomb group ring finger 2
ZBTB16     zinc finger and BTB domain containing 16
DCDC2C     doublecortin domain containing 2C
TRIM25     tripartite motif containing 25
ZNF

RNF122     ring finger protein 122
CFAP69     cilia and flagella associated protein 69
CMTM7      CKLF like MARVEL transmembrane domain containing 7
PDZD3      PDZ domain containing 3
NEK11      NIMA related kinase 11
ZNF669     zinc finger protein 669
RBFA       ribosome binding factor A (putative)
TREML2     triggering receptor expressed on myeloid cells like 2
BORA       bora, aurora kinase A activator
TCTN2      tectonic family member 2
CPSF7      cleavage and polyadenylation specific factor 7
RPAP2      RNA polymerase II associated protein 2
CBLL1      Cbl proto-oncogene like 1, E3 ubiquitin protein ligase
RABEP2     rabaptin, RAB GTPase binding effector protein 2
THSD4      thrombospondin type 1 domain containing 4
UBA5       ubiquitin like modifier activating enzyme 5
CCDC134    coiled-coil domain containing 134
ZC3H14     zinc finger CCCH-type containing 14
MAP9       microtubule associated protein 9
HDAC11     histone deacetylase 11
CAAP1      caspase activity and apoptosis in

CABIN1     calcineurin binding protein 1
SRRM2      serine/arginine repetitive matrix 2
SKA1       spindle and kinetochore associated complex subunit 1
RNF180     ring finger protein 180
SREK1IP1   SREK1 interacting protein 1
CLCF1      cardiotrophin-like cytokine factor 1
ZNF281     zinc finger protein 281
HMHA1      histocompatibility (minor) HA-1
IL36G      interleukin 36, gamma
SLC7A10    solute carrier family 7 member 10
TRPV5      transient receptor potential cation channel subfamily V member 5
PIK3R5     phosphoinositide-3-kinase regulatory subunit 5
NNT        nicotinamide nucleotide transhydrogenase
ADAT1      adenosine deaminase, tRNA-specific 1
OR52A1     olfactory receptor family 52 subfamily A member 1
SLC16A8    solute carrier family 16 member 8
PRAME      preferentially expressed antigen in melanoma
SEC14L2    SEC14 like lipid binding 2
MAPK8IP2   mitogen-activated protein kinase 8 interacting protein 2
RBFOX2     RNA binding protein, fox-1 homolog (C. elegans) 2
SEZ6L  

In [64]:
set(rbp_id['entrezgene']).intersection(geneids)

set()

In [66]:
len(set(rbp_id.index).intersection(symbols)) #good

141

# RNA polymerase

In [69]:
rbp_id[rbp_id.index.str.startswith('R')] # no RNA polymerase

,HGNC,MIM,_id,_score,alias,entrezgene,exons,exons_hg19,generif,interpro,...,go.BP.pubmed,go.CC.evidence,go.CC.gocategory,go.CC.id,go.CC.pubmed,go.CC.term,reagent.CondMedia_CM_LibrAB.id,reagent.CondMedia_CM_LibrAB.relationship,esnp,esnt
query,,,,,,,,,,,,,,,,,,,,,
RBFOX2,9906,612149,23543,87.636925,"[FOX2, Fox-2, HNRBP2, HRNBP2, RBM9, RTA, dJ106...",23543,"[{'cdsend': 161216, 'cdsstart': 65202, 'chr': ...","[{'cdsend': 36236265, 'cdsstart': 36140251, 'c...","[{'pubmed': 16260614, 'text': 'Fox-1 and Fox-2...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000475142, ENSP00000485711, ENSP0000048...","{ENST00000491982, ENST00000408983, ENST0000063..."
RBM22,25503,612430,55696,88.291320,"[Cwc2, ZC3H16, fSAP47]",55696,"[{'cdsend': 150700985, 'cdsstart': 150691750, ...","[{'cdsend': 150080547, 'cdsstart': 150071312, ...","[{'pubmed': 17045351, 'text': 'ALG-2 alone eve...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000199814, ENSP00000428154, ENSP0000041...","{ENST00000521248, ENST00000518917, ENST0000052..."
RBM5,9902,606884,10181,86.758950,"[G15, H37, LUCA-15, LUCA15, RMB5]",10181,"[{'cdsend': 50118456, 'cdsstart': 50090434, 'c...","[{'cdsend': 50155889, 'cdsstart': 50127867, 'c...","[{'pubmed': 12581154, 'text': 'Over-expression...","[{'desc': 'G-patch domain', 'id': 'IPR000467',...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000384872, ENSP00000406119, ENSP0000039...","{ENST00000441812, ENST00000464988, ENST0000046..."
RBM15,14959,606077,64783,87.279670,"[OTT, OTT1, SPEN]",64783,"[{'cdsend': 110346354, 'cdsstart': 110339405, ...","[{'cdsend': 110888976, 'cdsstart': 110882027, ...","[{'pubmed': 15849773, 'text': 'Fusion with MKL...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000498296, ENSP00000358799, ENSP0000049...","{ENST00000369784, ENST00000652747, ENST0000048..."
RPS3,10420,600454,6188,65.414340,S3,6188,"[{'cdsend': 75404865, 'cdsstart': 75399547, 'c...","[{'cdsend': 75115909, 'cdsstart': 75110591, 'c...","[{'pubmed': 11911468, 'text': 'Electron parama...","[{'desc': 'Ribosomal protein S3, C-terminal', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000433421, ENSP00000436556, ENSP0000043...","{ENST00000526248, ENST00000528439, ENST0000052..."
RPS11,10384,180471,6205,66.056090,S11,6205,"[{'cdsend': 49499635, 'cdsstart': 49496456, 'c...","[{'cdsend': 50002892, 'cdsstart': 49999713, 'c...","[{'pubmed': 26506620, 'text': 'increased expre...","[{'desc': 'Ribosomal protein S17/S11', 'id': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000470375, ENSP00000471873, ENSP0000047...","{ENST00000600027, ENST00000596873, ENST0000059..."


# identify ribosomal proteins

In [11]:
ribosome = pd.read_csv('protein_class_Ribosomal.tsv', header = 0, sep  = '\t')

In [12]:
# only two are ribosomal protein
rbp_id.loc[rbp_id['ensembl.gene'].isin(ribosome['Ensembl'])]

,HGNC,MIM,_id,_score,alias,entrezgene,exons,exons_hg19,generif,interpro,...,go.BP.pubmed,go.CC.evidence,go.CC.gocategory,go.CC.id,go.CC.pubmed,go.CC.term,reagent.CondMedia_CM_LibrAB.id,reagent.CondMedia_CM_LibrAB.relationship,esnp,esnt
query,,,,,,,,,,,,,,,,,,,,,
RPS3,10420,600454,6188,65.41434,S3,6188,"[{'cdsend': 75404865, 'cdsstart': 75399547, 'c...","[{'cdsend': 75115909, 'cdsstart': 75110591, 'c...","[{'pubmed': 11911468, 'text': 'Electron parama...","[{'desc': 'Ribosomal protein S3, C-terminal', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000433421, ENSP00000436556, ENSP0000043...","{ENST00000526248, ENST00000528439, ENST0000052..."
RPS11,10384,180471,6205,66.05609,S11,6205,"[{'cdsend': 49499635, 'cdsstart': 49496456, 'c...","[{'cdsend': 50002892, 'cdsstart': 49999713, 'c...","[{'pubmed': 26506620, 'text': 'increased expre...","[{'desc': 'Ribosomal protein S17/S11', 'id': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000470375, ENSP00000471873, ENSP0000047...","{ENST00000600027, ENST00000596873, ENST0000059..."


# identify splicesome protein

In [13]:
splicesome = pd.read_csv('splicesome_proteins.csv', header = 0)

In [14]:
rbp_id.loc[rbp_id['ensembl.gene'].isin(splicesome['Ensemble Gene ID'])]

,HGNC,MIM,_id,_score,alias,entrezgene,exons,exons_hg19,generif,interpro,...,go.BP.pubmed,go.CC.evidence,go.CC.gocategory,go.CC.id,go.CC.pubmed,go.CC.term,reagent.CondMedia_CM_LibrAB.id,reagent.CondMedia_CM_LibrAB.relationship,esnp,esnt
query,,,,,,,,,,,,,,,,,,,,,
HNRNPC,5035,164020,3183,89.639970,"[C1, C2, HNRNP, HNRPC, SNRPC]",3183,"[{'cdsend': 21234193, 'cdsstart': 21211222, 'c...","[{'cdsend': 21702352, 'cdsstart': 21679381, 'c...","[{'pubmed': 11877401, 'text': 'phosphorylation...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000450790, ENSP00000450725, ENSP0000045...","{ENST00000557768, ENST00000420743, ENST0000055..."
RBFOX2,9906,612149,23543,87.636925,"[FOX2, Fox-2, HNRBP2, HRNBP2, RBM9, RTA, dJ106...",23543,"[{'cdsend': 161216, 'cdsstart': 65202, 'chr': ...","[{'cdsend': 36236265, 'cdsstart': 36140251, 'c...","[{'pubmed': 16260614, 'text': 'Fox-1 and Fox-2...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000475142, ENSP00000485711, ENSP0000048...","{ENST00000491982, ENST00000408983, ENST0000063..."
IGF2BP1,28866,608288,10642,88.099540,"[CRD-BP, CRDBP, IMP-1, IMP1, VICKZ1, ZBP1]",10642,"[{'cdsend': 49049444, 'cdsstart': 48997745, 'c...","[{'cdsend': 47126806, 'cdsstart': 47075107, 'c...","[{'pubmed': 11973350, 'text': 'IMP1 localizati...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000290341, ENSP00000389135}","{ENST00000431824, ENST00000499130, ENST0000051..."
HNRNPK,5044,600712,3190,86.588570,"[AUKS, CSBP, HNRPK, TUNP]",3190,"[{'cdsend': 83978252, 'cdsstart': 83969349, 'c...","[{'cdsend': 86593167, 'cdsstart': 86584264, 'c...","[{'pubmed': 11867641, 'text': 'Heterogeneous n...","[{'desc': 'K Homology domain', 'id': 'IPR00408...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000473957, ENSP00000409456, ENSP0000047...","{ENST00000481820, ENST00000493362, ENST0000036..."
SRSF7,10789,600572,6432,90.235690,"[9G8, AAG3, SFRS7]",6432,"[{'cdsend': 38751256, 'cdsstart': 38745132, 'c...","[{'cdsend': 38978398, 'cdsstart': 38972274, 'c...","[{'pubmed': 15123677, 'text': 'SR proteins 9G8...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000402264, ENSP00000416516, ENSP0000032...","{ENST00000443213, ENST00000446327, ENST0000048..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PABPN1,8565,602279,8106,91.164940,"[OPMD, PAB2, PABII, PABP-2, PABP2]",8106,"[{'cdsend': 23324299, 'cdsstart': 23321469, 'c...","[{'cdsend': 23793508, 'cdsstart': 23790678, 'c...","[{'pubmed': 11222452, 'text': 'Observational s...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000450724, ENSP00000451592, ENSP0000038...","{ENST00000554062, ENST00000556821, ENST0000055..."
PRPF4,17349,607795,9128,87.269264,"[HPRP4, HPRP4P, PRP4, Prp4p, RP70, SNRNP60]",9128,"[{'cdsend': 113291660, 'cdsstart': 113275743, ...","[{'cdsend': 116053940, 'cdsstart': 116038023, ...","[{'pubmed': 11971898, 'text': 'interaction bet...","[{'desc': 'WD40 repeat', 'id': 'IPR001680', 's...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000363313, ENSP00000363315}","{ENST00000488937, ENST00000374199, ENST0000037..."
TIAL1,11804,603413,7073,86.114980,"[TCBP, TIAR]",7073,"[{'cdsend': 119596465, 'cdsstart': 119575664, ...","[{'cdsend': 121355977, 'cdsstart': 121335176, ...","[{'pubmed': 12917321, 'text': 'results report ...","[{'desc': 'RNA recognition motif domain', 'id'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{ENSP00000403573, ENSP00000394902, ENSP0000043...","{ENST00000495821, ENST00000369093, ENST0000036..."


In [15]:
rbp_id.to_pickle('RBP_id.pickle')